
# LeetCode 1071 — Greatest Common Divisor of Strings

**Problem statement.**  
Given two strings `str1` and `str2`, return *the largest string* `x` such that `x` divides both `str1` and `str2`.





A string `x` **divides** string `y` if and only if `y` is one or more concatenations of `x` (i.e., `y == x * k` for some integer `k ≥ 1`).  
If no such string exists, return the empty string `""`.

- **Input:** `str1: str`, `str2: str`  
- **Output:** `str` (the greatest common divisor string)  
- **Typical constraints:** `1 ≤ len(str1), len(str2) ≤ 1000` and `str1`, `str2` consist of uppercase English letters.

**Examples**
- `str1 = "ABCABC"`, `str2 = "ABC"` → `"ABC"`  
- `str1 = "ABABAB"`, `str2 = "ABAB"` → `"AB"`  
- `str1 = "LEET"`, `str2 = "CODE"` → `""`

**File Info**  
- **ID:** 1071  
- **short_title/slug:** `gcd_of_strings` / `greatest-common-divisor-of-strings`  
- **Bench input sizes:** `[1, 10, 100, 1000, 5000, 10000]`


# Stats

![Merge Example](./images/1071_gcd_of_strings.png)



## Solutions Overview

We'll implement and compare two approaches:

1. **Prefix Scan (Brute Force over Candidate Divisors)**  
   - Intuition: The GCD string must be a prefix of both strings. Try candidate prefixes (ideally only those whose lengths divide both `len(str1)` and `len(str2)`) from longest to shortest. The first candidate that tiles both strings is the answer.  
   - Complexity: In the worst case can approach **O(n²)** if implemented naïvely over many prefixes; restricting to *divisor lengths* makes it significantly faster in practice.

2. **Mathematical GCD with Concatenation Check**  
   - Property: If there exists a common divisor string, then `str1 + str2 == str2 + str1`. Otherwise, no solution.  
   - If the property holds, the answer is the prefix of length `gcd(len(str1), len(str2))` from either string.  
   - Complexity: **O(n)** to check concatenation equality; slicing the prefix is O(n) total.


In [1]:

from typing import *

def gcd_of_strings_bruteforce(str1: str, str2: str) -> str:
    """Prefix-scan solution that checks only divisor-length prefixes.

    We iterate over candidate lengths that divide both len(str1) and len(str2),
    from largest to smallest, and return the first prefix that tiles both strings.
    """
    n1, n2 = len(str1), len(str2)
    # Helper to get all common divisor lengths, largest first
    def common_divisor_lengths(a: int, b: int) -> List[int]:
        divs = []
        lim = min(a, b)
        for L in range(1, lim + 1):
            if a % L == 0 and b % L == 0:
                divs.append(L)
        divs.sort(reverse=True)
        return divs

    for L in common_divisor_lengths(n1, n2):
        prefix = str1[:L]
        if prefix * (n1 // L) == str1 and prefix * (n2 // L) == str2:
            return prefix
    return ""


In [2]:

from typing import *
from math import gcd

def gcd_of_strings_gcd(str1: str, str2: str) -> str:
    """Elegant GCD solution using the concatenation property.

    If str1 + str2 != str2 + str1, no gcd-string exists.
    Else, it is the prefix of length gcd(len(str1), len(str2)).
    """
    if str1 + str2 != str2 + str1:
        return ""
    size = gcd(len(str1), len(str2))
    return str1[:size]


In [3]:

# --- Correctness Tests ---

def run_basic_tests(func):
    cases = [
        ("ABCABC", "ABC", "ABC"),
        ("ABABAB", "ABAB", "AB"),
        ("LEET", "CODE", ""),
        ("AAAAAA", "AAA", "AAA"),
        ("ABCABCABC", "ABCABC", "ABC"),
        ("A", "A", "A"),
        ("ABCD", "ABCDABCD", "ABCD"),
        ("XYZ", "XYZXYZXYZXYZ", "XYZ"),
        ("ABAB", "ABBA", ""),
    ]
    for i, (s1, s2, expected) in enumerate(cases, 1):
        got = func(s1, s2)
        assert got == expected, f"Case {i} failed: {func.__name__}({s1!r}, {s2!r}) = {got!r}, expected {expected!r}"
    print(f"{func.__name__}: basic tests passed ({len(cases)} cases).")


def run_constructive_random_tests(func, *, alphabet="ABCD", base_len=3, max_rep=20):
    # Deterministic "random-ish" generation without RNG for reproducibility:
    # We'll cycle patterns and repetition counts programmatically.
    def make_base(i: int) -> str:
        s = []
        for k in range(base_len):
            s.append(alphabet[(i + k) % len(alphabet)])
        return "".join(s)

    # Positive cases: both are multiples of the same base
    for i in range(1, 21):
        base = make_base(i)
        r1 = 1 + (i % max_rep)
        r2 = 1 + ((2*i) % max_rep)
        s1, s2 = base * r1, base * r2
        expected = base * __import__("math").gcd(r1, r2)
        got = func(s1, s2)
        assert got == expected, f"Positive constructive failed for i={i}: got {got!r}, expected {expected!r}"

    # Negative cases: break one string by appending a different char
    for i in range(1, 21):
        base = make_base(i)
        r1 = 1 + (i % max_rep)
        r2 = 1 + ((3*i) % max_rep)
        s1, s2 = base * r1, (base * r2) + "X"  # force mismatch
        got = func(s1, s2)
        assert got == "", f"Negative constructive failed for i={i}: got {got!r}, expected ''"

    print(f"{func.__name__}: constructive tests passed.")


# Run tests for both implementations
run_basic_tests(gcd_of_strings_bruteforce)
run_basic_tests(gcd_of_strings_gcd)

run_constructive_random_tests(gcd_of_strings_bruteforce)
run_constructive_random_tests(gcd_of_strings_gcd)

print("All correctness tests passed.\n")


gcd_of_strings_bruteforce: basic tests passed (9 cases).
gcd_of_strings_gcd: basic tests passed (9 cases).
gcd_of_strings_bruteforce: constructive tests passed.
gcd_of_strings_gcd: constructive tests passed.
All correctness tests passed.



In [4]:

# --- Micro-benchmarks ---
import timeit
import pandas as pd

SIZES = [1, 10, 100, 1000, 5000, 10000]

def make_pair_match(n: int) -> tuple[str, str]:
    base = "ABC"
    s1 = (base * (n // len(base) + 1))[:n]
    s2 = (base * (2 * n // len(base) + 1))[:2*n]  # ~2x length, same pattern -> full GCD exists
    return s1, s2

def make_pair_mismatch(n: int) -> tuple[str, str]:
    base1, base2 = "ABC", "ABD"
    s1 = (base1 * (n // len(base1) + 1))[:n]
    s2 = (base2 * (n // len(base2) + 1))[:n]      # different pattern -> no GCD (except possibly tiny 'A')
    return s1, s2

def time_func(func, s1, s2, repeat=5):
    stmt = "func(s1, s2)"
    setup = "from __main__ import func, s1, s2"
    return min(timeit.Timer(stmt, setup=setup).repeat(repeat=repeat, number=1))

rows = []
for n in SIZES:
    s1m, s2m = make_pair_match(n)
    s1x, s2x = make_pair_mismatch(n)

    for label, a, b in [("match", s1m, s2m), ("mismatch", s1x, s2x)]:
        t_bf = time_func(gcd_of_strings_bruteforce, a, b)
        t_gd = time_func(gcd_of_strings_gcd, a, b)
        rows.append({
            "n_base": n,
            "case": label,
            "bruteforce_s": t_bf,
            "gcd_s": t_gd,
            "speedup_gcd_vs_bf": t_bf / t_gd if t_gd > 0 else float("inf"),
        })

df_bench = pd.DataFrame(rows)
df_pivot = df_bench.pivot(index=["n_base", "case"], columns=None)[["bruteforce_s", "gcd_s", "speedup_gcd_vs_bf"]]
from caas_jupyter_tools import display_dataframe_to_user
display_dataframe_to_user("GCD of Strings — Micro-benchmarks", df_pivot)
print("Benchmark table displayed above.")


ImportError: cannot import name 'func' from '__main__' (unknown location)


## Benchmark Takeaways

- The **GCD approach** is consistently faster and scales linearly with input size because it performs a single concatenation equality check plus slicing by `gcd(len1, len2)`.
- The **prefix-scan approach** improves markedly when we restrict checks to **divisor lengths** only, yet it still performs more checks in adversarial cases and can be noticeably slower.
- For production, prefer the **GCD solution** for clarity and performance.


In [ ]:

from typing import *
from math import gcd

class Solution:
    def gcdOfStrings(self, str1: str, str2: str) -> str:
        """Production version using the GCD property.

        Time: O(n) to compare concatenations and slice the prefix.
        Space: O(1) extra (ignoring Python's internal string allocations).
        """
        if str1 + str2 != str2 + str1:
            return ""
        size = gcd(len(str1), len(str2))
        return str1[:size]



## Appendix: Methods & Functions Explained

- **Concatenation Equality Check (`str1 + str2 == str2 + str1`)**  
  If two strings are made of repetitions of a common base pattern, concatenating them in either order yields identical results. If the concatenations differ, no common divisor string exists.

- **Greatest Common Divisor of Lengths**  
  When a common base pattern exists, its length must divide both `len(str1)` and `len(str2)`. Therefore its length equals `gcd(len(str1), len(str2))`.

- **Why Scanning Only Divisor Lengths Helps**  
  The GCD string length must be a divisor of both lengths. Restricting candidates to common divisors avoids trying obviously invalid lengths.


In [ ]:

import sys, platform, nbformat, math, pandas
print({"python": sys.version, "platform": platform.platform(), "nbformat": nbformat.__version__, "math": math.__doc__ is not None, "pandas": pandas.__version__})



## Notes

- **Problem Title:** Greatest Common Divisor of Strings  
- **ID:** 1071  
- **short_title/slug:** `gcd_of_strings` / `greatest-common-divisor-of-strings`  
- **Bench input sizes:** `[1, 10, 100, 1000, 5000, 10000]`

Coding style notes:
- Functions include **type hints** and avoid `+=` on strings in favor of `''.join(...)` where concatenation in loops would otherwise occur.
- In this problem, Python's string multiplication `prefix * k` is both idiomatic and efficient.
